In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/international-football-results-from-1872-to-2017/goalscorers.csv
/kaggle/input/international-football-results-from-1872-to-2017/shootouts.csv
/kaggle/input/international-football-results-from-1872-to-2017/results.csv
/kaggle/input/spelers-informatie/players.csv
/kaggle/input/spelers-informatie/competitions.csv
/kaggle/input/spelers-informatie/games.csv
/kaggle/input/spelers-informatie/game_events.csv
/kaggle/input/spelers-informatie/club_games.csv
/kaggle/input/spelers-informatie/player_valuations.csv
/kaggle/input/spelers-informatie/game_lineups.csv
/kaggle/input/spelers-informatie/appearances.csv
/kaggle/input/spelers-informatie/clubs.csv
/kaggle/input/fifaworldranking/fifa_ranking-2024-04-04.csv
/kaggle/input/fifaworldranking/fifa_ranking-2023-07-20.csv


# Inladen data

Lees de data in, voeg het jaartal van de wedstrijd toe en creëer een subset met de wedstrijden vanaf 2010

In [2]:
games = pd.read_csv("/kaggle/input/international-football-results-from-1872-to-2017/results.csv")

games['date'] = pd.to_datetime(games['date'], format='%Y-%m-%d')
games['year'] = games['date'].dt.year
games_since_2010 = games[games['date'] >= "2010-01-01"]

games_since_2010.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
33545,2010-01-02,Iran,North Korea,1.0,0.0,Friendly,Doha,Qatar,True,2010
33546,2010-01-02,Qatar,Mali,0.0,0.0,Friendly,Doha,Qatar,False,2010
33547,2010-01-02,Syria,Zimbabwe,6.0,0.0,Friendly,Kuala Lumpur,Malaysia,True,2010
33548,2010-01-02,Yemen,Tajikistan,0.0,1.0,Friendly,Sana'a,Yemen,False,2010
33549,2010-01-03,Angola,Gambia,1.0,1.0,Friendly,Vila Real de Santo António,Portugal,True,2010


# Definiëren functies

Maak drie functies voor het ophalen van recente wedstrijdgegevens. Per wedstrijd worden de meest recente wedstrijden van beide teams en de meest recente onderlinge wedstrijden opgehaald en wordt het gemiddelde aantal gescoorde goals en de win en loss rate berekend. Het aantal wedstrijden is variabel en wordt als argument meegegeven bij het aanroepen van de functies.

In [3]:
def get_stats(row, team):
    if row['home_team'] == team:
        return [row['home_score'], 1 if row['home_score'] > row['away_score'] else 0, 1 if row['home_score'] < row['away_score'] else 0]
    else:
        return [row['away_score'], 1 if row['away_score'] > row['home_score'] else 0, 1 if row['away_score'] < row['home_score'] else 0]

In [4]:
def get_recent_stats(row, n_matches, home_away):
    df = games.loc[((games['home_team'] == row[home_away]) | (games['away_team'] == row[home_away]))
                    & (games['date'] < row['date'])].sort_values('date', ascending=False).head(n_matches)
    if not df.empty:
        df[['team_score', 'team_win', 'team_loss']] = df.apply(get_stats, axis=1, args=(row[home_away],), result_type='expand')
    else:
        df[['team_score', 'team_win', 'team_loss']] = pd.DataFrame(columns=['team_score', 'team_win', 'team_loss'])
    return [df['team_score'].mean(), df['team_win'].mean(), df['team_loss'].mean()]

In [5]:
def get_head_to_head_stats(row, n_matches, home_away):
    df = games.loc[((games['home_team'] == row['home_team']) & (games['away_team'] == row['away_team'])
                    | (games['home_team'] == row['away_team']) & (games['away_team'] == row['home_team']))
                   & (games['date'] < row['date'])].sort_values('date', ascending=False).head(n_matches)
    if not df.empty:
        df[['team_score', 'team_win', 'team_loss']] = df.apply(get_stats, axis=1, args=(row[home_away],), result_type='expand')
    else:
        df[['team_score', 'team_win', 'team_loss']] = pd.DataFrame(columns=['team_score', 'team_win', 'team_loss'])
    return [df['team_score'].mean(), df['team_win'].mean(), df['team_loss'].mean()]

# Functies toepassen op dataset

De functies worden toegepast op de dataset met wedstrijden vanaf 2010 om voor elke wedstrijd kolommen toe te voegen met het gemiddelde aantal goals (ag), win rate (wr) en loss rate (lr) voor de recente wedstrijden (rec) en de onderlingen wedstrijden (h2h).

In [6]:
recent_matches = 5

# test_nl = games_since_2010.loc[(games_since_2010['home_team'] == "Netherlands") | (games_since_2010['away_team'] == "Netherlands")].drop(['tournament', 'city', 'country', 'neutral'], axis=1)
# test_nl = test_nl.head(20)
games_since_2010[['home_rec_ag', 'home_rec_wr', 'home_rec_lr']] = games_since_2010.apply(get_recent_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
games_since_2010[['away_rec_ag', 'away_rec_wr', 'away_rec_lr']] = games_since_2010.apply(get_recent_stats, axis=1, args=(recent_matches, 'away_team'), result_type='expand')
display(games_since_2010.head(10))

/tmp/ipykernel_18/3329019875.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_since_2010[['home_rec_ag', 'home_rec_wr', 'home_rec_lr']] = games_since_2010.apply(get_recent_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
/tmp/ipykernel_18/3329019875.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_since_2010[['home_rec_ag', 'home_rec_wr', 'home_rec_lr']] = games_since_2010.apply(get_recent_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
/tmp

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,home_rec_ag,home_rec_wr,home_rec_lr,away_rec_ag,away_rec_wr,away_rec_lr
33545,2010-01-02,Iran,North Korea,1.0,0.0,Friendly,Doha,Qatar,True,2010,1.0,0.2,0.6,2.2,0.6,0.2
33546,2010-01-02,Qatar,Mali,0.0,0.0,Friendly,Doha,Qatar,False,2010,1.4,0.4,0.4,1.2,0.4,0.2
33547,2010-01-02,Syria,Zimbabwe,6.0,0.0,Friendly,Kuala Lumpur,Malaysia,True,2010,0.6,0.2,0.4,1.4,0.4,0.2
33548,2010-01-02,Yemen,Tajikistan,0.0,1.0,Friendly,Sana'a,Yemen,False,2010,1.2,0.6,0.2,1.4,0.4,0.6
33549,2010-01-03,Angola,Gambia,1.0,1.0,Friendly,Vila Real de Santo António,Portugal,True,2010,0.2,0.0,0.4,1.0,0.4,0.2
33550,2010-01-04,Egypt,Mali,1.0,0.0,Friendly,Dubai,United Arab Emirates,True,2010,1.8,0.6,0.2,1.0,0.4,0.2
33551,2010-01-04,Tanzania,Ivory Coast,0.0,1.0,Friendly,Dar es Salaam,Tanzania,False,2010,1.4,0.6,0.4,2.2,0.4,0.0
33552,2010-01-05,Ghana,Malawi,0.0,0.0,Friendly,Manzini,Swaziland,True,2010,1.0,0.0,0.6,0.6,0.2,0.4
33553,2010-01-06,Bahrain,Hong Kong,4.0,0.0,AFC Asian Cup qualification,Riffa,Bahrain,False,2010,2.2,0.4,0.2,2.4,0.2,0.6
33554,2010-01-06,Benin,Libya,1.0,0.0,Friendly,Cotonou,Benin,False,2010,1.2,0.4,0.2,2.0,0.4,0.4


In [7]:
head_to_head_matches = 3

games_since_2010[['home_h2h_ag', 'home_h2h_wr', 'home_h2h_lr']] = games_since_2010.apply(get_head_to_head_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
games_since_2010[['away_h2h_ag', 'away_h2h_wr', 'away_h2h_lr']] = games_since_2010.apply(get_head_to_head_stats, axis=1, args=(recent_matches, 'away_team'), result_type='expand')
display(games_since_2010.head(10))

/tmp/ipykernel_18/726697471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_since_2010[['home_h2h_ag', 'home_h2h_wr', 'home_h2h_lr']] = games_since_2010.apply(get_head_to_head_stats, axis=1, args=(recent_matches, 'home_team'), result_type='expand')
/tmp/ipykernel_18/726697471.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_since_2010[['home_h2h_ag', 'home_h2h_wr', 'home_h2h_lr']] = games_since_2010.apply(get_head_to_head_stats, axis=1, args=(recent_matches, 'home_team'), result_type='exp

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,home_rec_lr,away_rec_ag,away_rec_wr,away_rec_lr,home_h2h_ag,home_h2h_wr,home_h2h_lr,away_h2h_ag,away_h2h_wr,away_h2h_lr
33545,2010-01-02,Iran,North Korea,1.0,0.0,Friendly,Doha,Qatar,True,2010,...,0.6,2.2,0.6,0.2,1.600000,0.800000,0.000000,0.2,0.000000,0.800000
33546,2010-01-02,Qatar,Mali,0.0,0.0,Friendly,Doha,Qatar,False,2010,...,0.4,1.2,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN
33547,2010-01-02,Syria,Zimbabwe,6.0,0.0,Friendly,Kuala Lumpur,Malaysia,True,2010,...,0.4,1.4,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN
33548,2010-01-02,Yemen,Tajikistan,0.0,1.0,Friendly,Sana'a,Yemen,False,2010,...,0.2,1.4,0.4,0.6,2.000000,1.000000,0.000000,1.0,0.000000,1.000000
33549,2010-01-03,Angola,Gambia,1.0,1.0,Friendly,Vila Real de Santo António,Portugal,True,2010,...,0.4,1.0,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN
33550,2010-01-04,Egypt,Mali,1.0,0.0,Friendly,Dubai,United Arab Emirates,True,2010,...,0.2,1.0,0.4,0.2,0.800000,0.400000,0.600000,1.0,0.600000,0.400000
33551,2010-01-04,Tanzania,Ivory Coast,0.0,1.0,Friendly,Dar es Salaam,Tanzania,False,2010,...,0.4,2.2,0.4,0.0,1.000000,0.000000,0.000000,1.0,0.000000,0.000000
33552,2010-01-05,Ghana,Malawi,0.0,0.0,Friendly,Manzini,Swaziland,True,2010,...,0.6,0.6,0.2,0.4,4.666667,1.000000,0.000000,0.0,0.000000,1.000000
33553,2010-01-06,Bahrain,Hong Kong,4.0,0.0,AFC Asian Cup qualification,Riffa,Bahrain,False,2010,...,0.2,2.4,0.2,0.6,2.333333,0.666667,0.333333,1.0,0.333333,0.666667
33554,2010-01-06,Benin,Libya,1.0,0.0,Friendly,Cotonou,Benin,False,2010,...,0.2,2.0,0.4,0.4,1.000000,0.500000,0.500000,2.0,0.500000,0.500000


*Sla de resultaten op, aangezien het toepassen van de functies een tijdje duurt*

In [8]:
games_since_2010.to_csv('games_since_2010.csv', index=False)

# Inladen dataset inclusief recente en onderlinge resultaten

Wanneer je niets aan de input hoeft te veranderen kun je hier beginnen

In [9]:
games_since_2010 = pd.read_csv('/kaggle/working/games_since_2010.csv')
display(games_since_2010.head())

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,home_rec_lr,away_rec_ag,away_rec_wr,away_rec_lr,home_h2h_ag,home_h2h_wr,home_h2h_lr,away_h2h_ag,away_h2h_wr,away_h2h_lr
0,2010-01-02,Iran,North Korea,1.0,0.0,Friendly,Doha,Qatar,True,2010,...,0.6,2.2,0.6,0.2,1.6,0.8,0.0,0.2,0.0,0.8
1,2010-01-02,Qatar,Mali,0.0,0.0,Friendly,Doha,Qatar,False,2010,...,0.4,1.2,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-01-02,Syria,Zimbabwe,6.0,0.0,Friendly,Kuala Lumpur,Malaysia,True,2010,...,0.4,1.4,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-01-02,Yemen,Tajikistan,0.0,1.0,Friendly,Sana'a,Yemen,False,2010,...,0.2,1.4,0.4,0.6,2.0,1.0,0.0,1.0,0.0,1.0
4,2010-01-03,Angola,Gambia,1.0,1.0,Friendly,Vila Real de Santo António,Portugal,True,2010,...,0.4,1.0,0.4,0.2,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
fifaranking = pd.read_csv('/kaggle/input/fifaworldranking/fifa_ranking-2024-04-04.csv')
# transform rank_date to year
fifaranking['rank_date'] = fifaranking['rank_date'].str[:4]
fifaranking['rank_date'].unique() # checking the values
fifaranking = fifaranking.rename(columns={'rank_date': 'year'})
fifaranking.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,year
0,83.0,Guatemala,GUA,15.0,0.0,83,CONCACAF,1992
1,32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992
2,33.0,Portugal,POR,38.0,0.0,33,UEFA,1992
3,34.0,Austria,AUT,38.0,0.0,34,UEFA,1992
4,35.0,Colombia,COL,36.0,0.0,35,CONMEBOL,1992


In [11]:
# drop unnecessary columns
fifaranking = fifaranking[['rank', 'country_full', 'country_abrv', 'total_points', 'confederation', 'year']]
fifaranking.head()

,rank,country_full,country_abrv,total_points,confederation,year
0,83.0,Guatemala,GUA,15.0,CONCACAF,1992
1,32.0,Zambia,ZAM,38.0,CAF,1992
2,33.0,Portugal,POR,38.0,UEFA,1992
3,34.0,Austria,AUT,38.0,UEFA,1992
4,35.0,Colombia,COL,36.0,CONMEBOL,1992


In [12]:
# calculate mean rank and total_points for each country per year
annual = fifaranking.groupby(['year', 'country_full'])[['rank', 'total_points']].agg('mean')
fifaranking_annual = pd.DataFrame(annual.reset_index())
fifaranking_annual.head()

,year,country_full,rank,total_points
0,1992,Albania,86.0,14.0
1,1992,Algeria,30.0,39.0
2,1992,Angola,102.0,10.0
3,1992,Antigua and Barbuda,109.0,8.0
4,1992,Argentina,10.0,51.0


In [13]:
# make new column calculating the distance between annual ranks for each country
fifaranking_annual['annual_distance_rank'] = fifaranking_annual.groupby('country_full')['rank'].diff().fillna(0).round(1)
fifaranking_annual.head()

,year,country_full,rank,total_points,annual_distance_rank
0,1992,Albania,86.0,14.0,0.0
1,1992,Algeria,30.0,39.0,0.0
2,1992,Angola,102.0,10.0,0.0
3,1992,Antigua and Barbuda,109.0,8.0,0.0
4,1992,Argentina,10.0,51.0,0.0


In [14]:
# make new column calculating the running minimum (highest rank)
fifaranking_annual = fifaranking_annual.sort_values(by=['country_full', 'year'])
def rolling_minimum(group):
    group['highest_rank_rolling'] = group['rank'].rolling(window=len(group), min_periods=1).min()
    return group
fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_minimum)
fifaranking_annual.head()

/tmp/ipykernel_18/1881609304.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_minimum)


year country_full        rank  total_points  \
country_full                                                     
Afghanistan  2079  2003  Afghanistan  199.000000     42.833333   
             2282  2004  Afghanistan  197.500000     62.750000   
             2486  2005  Afghanistan  198.833333     57.750000   
             2691  2006  Afghanistan  181.454545     67.363636   
             2898  2007  Afghanistan  186.250000     24.333333   

                   annual_distance_rank  highest_rank_rolling  
country_full                                                   
Afghanistan  2079                   0.0            199.000000  
             2282                  -1.5            197.500000  
             2486                   1.3            197.500000  
             2691                 -17.4            181.454545  
             2898                   4.8            181.454545

In [15]:
# make new column calculating the distance between annual points for each country
fifaranking_annual['annual_distance_points'] = fifaranking_annual['total_points'].diff().fillna(0).round(1)
fifaranking_annual.head()

year country_full        rank  total_points  \
country_full                                                     
Afghanistan  2079  2003  Afghanistan  199.000000     42.833333   
             2282  2004  Afghanistan  197.500000     62.750000   
             2486  2005  Afghanistan  198.833333     57.750000   
             2691  2006  Afghanistan  181.454545     67.363636   
             2898  2007  Afghanistan  186.250000     24.333333   

                   annual_distance_rank  highest_rank_rolling  \
country_full                                                    
Afghanistan  2079                   0.0            199.000000   
             2282                  -1.5            197.500000   
             2486                   1.3            197.500000   
             2691                 -17.4            181.454545   
             2898                   4.8            181.454545   

                   annual_distance_points  
country_full                               
Afghanistan  2079                     0.0  
             2282                    19.9  
             2486                    -5.0  
             2691                     9.6  
             2898                   -43.0

In [16]:
# rolling average rank per country per year
def rolling_avg(group):
    group['avg_rank_rolling'] = group['rank'].rolling(window=len(group), min_periods=1).mean().round(1)
    return group
fifaranking_annual.reset_index(drop=True, inplace=True)
fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_avg)
fifaranking_annual.head()

/tmp/ipykernel_18/3422628461.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fifaranking_annual = fifaranking_annual.groupby('country_full').apply(rolling_avg)


year country_full        rank  total_points  \
country_full                                                  
Afghanistan  0  2003  Afghanistan  199.000000     42.833333   
             1  2004  Afghanistan  197.500000     62.750000   
             2  2005  Afghanistan  198.833333     57.750000   
             3  2006  Afghanistan  181.454545     67.363636   
             4  2007  Afghanistan  186.250000     24.333333   

                annual_distance_rank  highest_rank_rolling  \
country_full                                                 
Afghanistan  0                   0.0            199.000000   
             1                  -1.5            197.500000   
             2                   1.3            197.500000   
             3                 -17.4            181.454545   
             4                   4.8            181.454545   

                annual_distance_points  avg_rank_rolling  
country_full                                              
Afghanistan  0                     0.0             199.0  
             1                    19.9             198.2  
             2                    -5.0             198.4  
             3                     9.6             194.2  
             4                   -43.0             192.6

In [17]:
#table order
fifaranking_annual = fifaranking_annual[['country_full', 'year', 'rank', 'annual_distance_rank', 'avg_rank_rolling', 'highest_rank_rolling', 'total_points', 'annual_distance_points']]
fifaranking_annual = fifaranking_annual.reset_index(drop=True)
fifaranking_annual.head()

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points
0,Afghanistan,2003,199.000000,0.0,199.0,199.000000,42.833333,0.0
1,Afghanistan,2004,197.500000,-1.5,198.2,197.500000,62.750000,19.9
2,Afghanistan,2005,198.833333,1.3,198.4,197.500000,57.750000,-5.0
3,Afghanistan,2006,181.454545,-17.4,194.2,181.454545,67.363636,9.6
4,Afghanistan,2007,186.250000,4.8,192.6,181.454545,24.333333,-43.0


In [18]:
lands = ["Germany", "Scotland", "Hungary", "Switzerland", "Spain", "Croatia", "Italy", "Albania", 
         "England", "Denmark", "Serbia", "Slovenia", "France", "Netherlands", "Austria", "Poland", 
         "Belgium", "Slovakia", "Romania", "Ukraine", "Turkey", "Portugal", "Czech Republic", "Georgia"]

## player valuation inladen; date goedzetten; jaar extraheren
Transfer_values = pd.read_csv("/kaggle/input/spelers-informatie/player_valuations.csv")
Transfer_values["date"] = pd.to_datetime(Transfer_values["date"])
Transfer_values['year'], Transfer_values['month'] = Transfer_values['date'].dt.year, Transfer_values['date'].dt.month

## groeperen op sleper en jaar; bereken het gemiddelde tranfser waarde
Transfer_value_players = Transfer_values.groupby(["player_id", "year"])["market_value_in_eur"].mean()
Transfer_value_players = pd.DataFrame(Transfer_value_players).reset_index()

## player informatie inladen; belangrijke kolommen houden; samenvoegen met Transfer_value_players; behoud alleen 2007+ en EK landen
players = pd.read_csv("/kaggle/input/spelers-informatie/players.csv")
players = players[["player_id","name", "country_of_citizenship"]]
Transfer_player_country = players.merge(Transfer_value_players, on = 'player_id')
Transfer_player_country = Transfer_player_country[(Transfer_player_country["year"] >=2007)&(Transfer_player_country['country_of_citizenship'].isin(lands))]
Country_transfer = pd.DataFrame(Transfer_player_country.groupby(["country_of_citizenship","year"])["market_value_in_eur"].mean()).reset_index()

# Groepeer de data per land en per jaar
grouped = Country_transfer.groupby(['country_of_citizenship', 'year'])

# Bereken het gemiddelde van de top 25 duurste spelers per groep; maak het een geheel getal van
landenlijst = grouped.apply(lambda x: x.nlargest(25, 'market_value_in_eur')['market_value_in_eur'].mean()).reset_index(name='gemiddelde_waarde')
landenlijst["gemiddelde_waarde"] = landenlijst["gemiddelde_waarde"].astype('int64')
landenlijst

/tmp/ipykernel_18/2568193831.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  landenlijst = grouped.apply(lambda x: x.nlargest(25, 'market_value_in_eur')['market_value_in_eur'].mean()).reset_index(name='gemiddelde_waarde')


,country_of_citizenship,year,gemiddelde_waarde
0,Albania,2007,1823484
1,Albania,2008,1776785
2,Albania,2009,1498437
3,Albania,2010,1032833
4,Albania,2011,776190
...,...,...,...
426,Ukraine,2020,520044
427,Ukraine,2021,563031
428,Ukraine,2022,595416
429,Ukraine,2023,746700


In [19]:
#transform to Czech Republic
df = fifaranking_annual[fifaranking_annual['country_full'].isin(lands)]
fifaranking_annual['country_full'] = np.where(fifaranking_annual['country_full'] == 'Czechia', "Czech Republic", fifaranking_annual['country_full'])
fifaranking_annual['country_full'].unique()

#merge tables
fifaranking_annual['year'] = fifaranking_annual['year'].astype('int64')
EKlanden = fifaranking_annual.merge(landenlijst, left_on=['country_full', 'year'], right_on=['country_of_citizenship', 'year'])
EKlanden

,country_full,year,rank,annual_distance_rank,avg_rank_rolling,highest_rank_rolling,total_points,annual_distance_points,country_of_citizenship,gemiddelde_waarde
0,Albania,2007,77.333333,-3.3,92.4,77.333333,414.583333,-46.6,Albania,1823484
1,Albania,2008,84.500000,7.2,91.9,77.333333,398.166667,-16.4,Albania,1776785
2,Albania,2009,90.416667,5.9,91.8,77.333333,375.833333,-22.3,Albania,1498437
3,Albania,2010,75.545455,-14.9,91.0,75.545455,435.636364,59.8,Albania,1032833
4,Albania,2011,64.083333,-11.5,89.6,64.083333,501.333333,65.7,Albania,776190
...,...,...,...,...,...,...,...,...,...,...
426,Ukraine,2020,23.875000,-1.2,45.3,16.666667,1534.375000,18.5,Ukraine,520044
427,Ukraine,2021,25.000000,1.1,44.6,16.666667,1523.942500,-10.4,Ukraine,563031
428,Ukraine,2022,26.833333,1.8,44.0,16.666667,1538.286667,14.3,Ukraine,595416
429,Ukraine,2023,24.000000,-2.8,43.4,16.666667,1543.855714,5.6,Ukraine,746700
